In [1]:
%reload_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim

if torch.cuda.is_available():
    print("Yeah we have a GPU!")
dev = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

import torchvision
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.utils import save_image
from torchvision.datasets import MNIST
import os

import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

import itertools
import time
import gc

from gdeep.decision_boundary import GradientFlowDecisionBoundaryCalculator, QuasihyperbolicDecisionBoundaryCalculator

Yeah we have a GPU!


# Train CNN-Autoencoder

In [4]:
# https://github.com/L1aoXingyu/pytorch-beginner/blob/master/08-AutoEncoder/conv_autoencoder.py


if not os.path.exists('./mlp_img'):
    os.mkdir('./mlp_img')


def to_img(x):
    x = 0.5 * (x + 1)
    x = x.clamp(0, 1)
    x = x.view(x.size(0), 1, 28, 28)
    return x


num_epochs = 100
batch_size = 128
learning_rate = 1e-3

img_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

dataset = MNIST('./data', transform=img_transform, download=True)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)


class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(28 * 28, 128),
            nn.ReLU(True),
            nn.Linear(128, 64),
            nn.ReLU(True),
            nn.Linear(64, 12),
            nn.ReLU(True),
            nn.Linear(12, 3))
        self.decoder = nn.Sequential(
            nn.Linear(3, 12),
            nn.ReLU(True),
            nn.Linear(12, 64),
            nn.ReLU(True),
            nn.Linear(64, 128),
            nn.ReLU(True), nn.Linear(128, 28 * 28), nn.Tanh())

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x


model_auto = autoencoder().to(dev)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(
    model_auto.parameters(), lr=learning_rate, weight_decay=1e-5)

In [4]:
"""
for epoch in range(num_epochs):
    for data in dataloader:
        img, _ = data
        img = img.view(img.size(0), -1)
        img = Variable(img).to(dev)
        # ===================forward=====================
        output = model_auto(img)
        loss = criterion(output, img)
        # ===================backward====================
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # ===================log========================
    print('epoch [{}/{}], loss:{:.4f}'
          .format(epoch + 1, num_epochs, loss.data.item()))
    if epoch % 10 == 0:
        pic = to_img(output.cpu().data)
        save_image(pic, 'mlp_img/image_{}.png'.format(epoch))

torch.save(model_auto, 'sim_autoencoder.pth')
"""

"\nfor epoch in range(num_epochs):\n    for data in dataloader:\n        img, _ = data\n        img = img.view(img.size(0), -1)\n        img = Variable(img).to(dev)\n        # ===================forward=====================\n        output = model_auto(img)\n        loss = criterion(output, img)\n        # ===================backward====================\n        optimizer.zero_grad()\n        loss.backward()\n        optimizer.step()\n    # ===================log========================\n    print('epoch [{}/{}], loss:{:.4f}'\n          .format(epoch + 1, num_epochs, loss.data.item()))\n    if epoch % 10 == 0:\n        pic = to_img(output.cpu().data)\n        save_image(pic, 'mlp_img/image_{}.png'.format(epoch))\n\ntorch.save(model_auto, 'sim_autoencoder.pth')\n"

In [5]:
model_auto = torch.load('sim_autoencoder.pth')
model_auto.eval()

autoencoder(
  (encoder): Sequential(
    (0): Linear(in_features=784, out_features=128, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): ReLU(inplace=True)
    (4): Linear(in_features=64, out_features=12, bias=True)
    (5): ReLU(inplace=True)
    (6): Linear(in_features=12, out_features=3, bias=True)
  )
  (decoder): Sequential(
    (0): Linear(in_features=3, out_features=12, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=12, out_features=64, bias=True)
    (3): ReLU(inplace=True)
    (4): Linear(in_features=64, out_features=128, bias=True)
    (5): ReLU(inplace=True)
    (6): Linear(in_features=128, out_features=784, bias=True)
    (7): Tanh()
  )
)

In [7]:
embedding_df = pd.DataFrame(columns=list('xyzc'))

for x, y in itertools.islice(dataloader, 20):
    embedded_batch = model_auto.encoder(x.view(x.size(0), -1)).detach().numpy()
    embedded_batch_df = pd.DataFrame(
        np.concatenate((embedded_batch, y.unsqueeze(1).numpy()), 1),
        columns=list('xyzc'))
    embedding_df = embedding_df.append(
        embedded_batch_df,
        ignore_index=True)

In [7]:
"""
fig = px.scatter_3d(embedding_df, x='x', y='y', z='z',
              color='c')
fig.show()
"""

"\nfig = px.scatter_3d(embedding_df, x='x', y='y', z='z',\n              color='c')\nfig.show()\n"

In [8]:
# https://github.com/pytorch/examples/blob/master/mnist/main.py
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR


transform=transforms.Compose([
    transforms.ToTensor(),
    #transforms.Normalize((0.1307,), (0.3081,))
    ])

batch_size = 64
test_batch_size = 64
train_kwargs = {'batch_size': batch_size}
test_kwargs = {'batch_size': test_batch_size}

dataset1 = datasets.MNIST('./data', train=True, download=True,
                    transform=transform)
dataset2 = datasets.MNIST('./data', train=False,
                    transform=transform)
train_loader = torch.utils.data.DataLoader(dataset1,**train_kwargs)
test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output


def train(log_interval, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [9]:
"""
seed = 42
torch.manual_seed(seed)
batch_size = 64
test_batch_size = 64
lr = 0.01
gamma = 0.7
epochs = 14
log_interval = 10

train_kwargs = {'batch_size': batch_size}
test_kwargs = {'batch_size': test_batch_size}

transform=transforms.Compose([
    transforms.ToTensor()
    ])


model = Net().to(dev)
optimizer = optim.Adadelta(model.parameters(), lr=lr)

scheduler = StepLR(optimizer, step_size=1, gamma=gamma)
for epoch in range(1, epochs + 1):
    train(log_interval, model, dev, train_loader, optimizer, epoch)
    test(model, dev, test_loader)
    scheduler.step()
"""

"\nseed = 42\ntorch.manual_seed(seed)\nbatch_size = 64\ntest_batch_size = 64\nlr = 0.01\ngamma = 0.7\nepochs = 14\nlog_interval = 10\n\ntrain_kwargs = {'batch_size': batch_size}\ntest_kwargs = {'batch_size': test_batch_size}\n\ntransform=transforms.Compose([\n    transforms.ToTensor()\n    ])\n\n\nmodel = Net().to(dev)\noptimizer = optim.Adadelta(model.parameters(), lr=lr)\n\nscheduler = StepLR(optimizer, step_size=1, gamma=gamma)\nfor epoch in range(1, epochs + 1):\n    train(log_interval, model, dev, train_loader, optimizer, epoch)\n    test(model, dev, test_loader)\n    scheduler.step()\n"

In [9]:
#torch.save(model, '/home/reinauer/EPFL/giotto-deep/examples/cnn_minst_10.pth')
model = torch.load('cnn_minst_10.pth')
model.eval()

Net(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (dropout1): Dropout(p=0.25, inplace=False)
  (dropout2): Dropout(p=0.5, inplace=False)
  (fc1): Linear(in_features=9216, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)

In [10]:
class LambdaLayer(nn.Module):
    def __init__(self, lam):
        super().__init__()
        self.lam = lam
    def forward(self, x):
        return self.lam(x)

class DBMulticlass(nn.Module):
    def forward(self, x):
        y = torch.topk(x, 2).values
        return (y[:, 0] - y[:, 1]) + 0.5

embedding_model = nn.Sequential(model_auto.decoder,
                    LambdaLayer(lambda x: x.reshape(-1, 1, 28, 28)),
                    model,
                    DBMulticlass())

In [12]:
"""
n_delta = 20
t = np.linspace(-10.5, 10.5, n_delta)
x, y, z = np.meshgrid(t,t,t)
xyz = torch.from_numpy(np.stack((x, y, z), axis=-1)).reshape(-1, 3)
xyz = xyz.type(torch.float32)
values = embedding_model(xyz).reshape(n_delta, n_delta, n_delta,-1).detach()
   
trace_1 = go.Isosurface(
    x = x.flatten(),
    y = y.flatten(),
    z = z.flatten(),
    value=values.flatten(),
    isomin=0.0,
    isomax=0.1)

fig = go.FigureWidget([trace_1])
fig.show()
"""

'\nn_delta = 20\nt = np.linspace(-10.5, 10.5, n_delta)\nx, y, z = np.meshgrid(t,t,t)\nxyz = torch.from_numpy(np.stack((x, y, z), axis=-1)).reshape(-1, 3)\nxyz = xyz.type(torch.float32)\nvalues = embedding_model(xyz).reshape(n_delta, n_delta, n_delta,-1).detach()\n   \ntrace_1 = go.Isosurface(\n    x = x.flatten(),\n    y = y.flatten(),\n    z = z.flatten(),\n    value=values.flatten(),\n    isomin=0.0,\n    isomax=0.1)\n\nfig = go.FigureWidget([trace_1])\nfig.show()\n'

In [11]:
torch.cuda.empty_cache()

embedding_model.to(dev)
embedding_model.eval()

Sequential(
  (0): Sequential(
    (0): Linear(in_features=3, out_features=12, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=12, out_features=64, bias=True)
    (3): ReLU(inplace=True)
    (4): Linear(in_features=64, out_features=128, bias=True)
    (5): ReLU(inplace=True)
    (6): Linear(in_features=128, out_features=784, bias=True)
    (7): Tanh()
  )
  (1): LambdaLayer()
  (2): Net(
    (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
    (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
    (dropout1): Dropout(p=0.25, inplace=False)
    (dropout2): Dropout(p=0.5, inplace=False)
    (fc1): Linear(in_features=9216, out_features=128, bias=True)
    (fc2): Linear(in_features=128, out_features=10, bias=True)
  )
  (3): DBMulticlass()
)

In [12]:
#del initial_points_batch
torch.cuda.empty_cache()

In [13]:
n_steps = 1000
precision = 0.01
range_db = 70.

n_batches = 2#60*12#100
batch_size = 200#1500#4000

# Initialize DataFrame
db_sample_df = pd.DataFrame(columns = ["x"+str(i) for i in range(3)])

for b in range(n_batches):
    tic = time.perf_counter()
    # Sample points on decision boundary
    initial_points_batch = range_db * torch.rand((batch_size, 3)) - range_db/2.
    initial_points_batch = initial_points_batch.to(dev, non_blocking=True)
    g = GradientFlowDecisionBoundaryCalculator(
                model=embedding_model,
                initial_points=initial_points_batch,
                optimizer=lambda params: torch.optim.Adam(params)
    )
    g.step(n_steps)
    
    # Save computed points to DataFrame
    db_sample_tensor = g.get_filtered_decision_boundary(precision)
    
    median = torch.median(embedding_model(db_sample_tensor) - 0.5).cpu().detach().numpy()
    db_sample = db_sample_tensor.to(device='cpu').detach().numpy()
    db_sample_df_batch = pd.DataFrame(db_sample, columns = ["x"+str(i) for i in range(3)])
    
    db_sample_df = db_sample_df.append(db_sample_df_batch, ignore_index=True)
    
    # Delete tensors on cuda
    del db_sample_tensor
    del initial_points_batch
    del db_sample
    torch.cuda.empty_cache()
    toc = time.perf_counter()
    print(f'batch: {b+1}/{n_batches} \t\
    computation time/batch: {toc - tic:0.4f}sec\t\
    number of points: {db_sample_df_batch.shape[0]}\n\t\
    median distance2db: {median}')
    del db_sample_df_batch

batch: 1/2 	    computation time/batch: 9.4228sec	    number of points: 31
	    median distance2db: 3.838539123535156e-05


KeyboardInterrupt: 

In [15]:
n_steps = 100
precision = 0.01
range_db = 70.

n_batches = 1000
batch_size = 3000#4000

# Initialize DataFrame
db_sample_df = pd.DataFrame(columns = ["x"+str(i) for i in range(3)])

for b in range(n_batches):
    tic = time.perf_counter()
    # Sample points on decision boundary
    initial_points_batch = range_db * torch.rand((batch_size, 3)) - range_db/2.
    initial_points_batch = initial_points_batch.to(dev, non_blocking=True)
    
    initial_points_batch.requires_grad = False
    delta = torch.zeros_like(initial_points_batch, requires_grad=True).to(dev)
    loss = embedding_model(initial_points_batch + delta).sum()
    loss.backward()
    initial_vectors = delta.grad.detach()
    
    
    g = QuasihyperbolicDecisionBoundaryCalculator(
                model=embedding_model,
                initial_points=initial_points_batch,
                initial_vectors=initial_vectors,
                integrator=None
    )
    g.step(n_steps)
    
    # Save computed points to DataFrame
    db_sample_tensor = g.get_filtered_decision_boundary(precision)
    toc = time.perf_counter()
    print(f'computation DB time/batch: {toc - tic:0.4f}')
    
    median = torch.median(embedding_model(db_sample_tensor) - 0.5).cpu().detach().numpy()
    db_sample = db_sample_tensor.to(device='cpu').detach().numpy()
    db_sample_df_batch = pd.DataFrame(db_sample, columns = ["x"+str(i) for i in range(3)])
    
    db_sample_df = db_sample_df.append(db_sample_df_batch, ignore_index=True)
    
    # Delete tensors on cuda
    del db_sample_tensor
    del initial_points_batch
    del db_sample
    toc = time.perf_counter()
    print(f'batch: {b+1}/{n_batches} \t\
    computation time/batch: {toc - tic:0.4f}sec\t\
    number of points: {db_sample_df_batch.shape[0]}\n\t\
    median distance2db: {median}')
    del db_sample_df_batch
    del initial_vectors
    del g
    del delta
    torch.cuda.empty_cache()
    gc.collect()
    if b%50 == 0:
        db_sample_df.to_csv(f'computed_points_1M_{b:04d}.csv')

computation DB time/batch: 56.3714
batch: 1/1000 	    computation time/batch: 56.3980sec	    number of points: 859
	    median distance2db: 1.1324882507324219e-05
computation DB time/batch: 55.8976
batch: 2/1000 	    computation time/batch: 55.9126sec	    number of points: 851
	    median distance2db: 1.3828277587890625e-05
computation DB time/batch: 56.1642
batch: 3/1000 	    computation time/batch: 56.1788sec	    number of points: 853
	    median distance2db: 9.775161743164062e-06
computation DB time/batch: 55.9077
batch: 4/1000 	    computation time/batch: 55.9231sec	    number of points: 895
	    median distance2db: 1.1444091796875e-05
computation DB time/batch: 55.9013
batch: 5/1000 	    computation time/batch: 55.9170sec	    number of points: 896
	    median distance2db: 1.1920928955078125e-05
computation DB time/batch: 55.9706
batch: 6/1000 	    computation time/batch: 55.9861sec	    number of points: 895
	    median distance2db: 1.0967254638671875e-05
computation DB time/batch:

computation DB time/batch: 56.1271
batch: 52/1000 	    computation time/batch: 56.1431sec	    number of points: 904
	    median distance2db: 1.0371208190917969e-05
computation DB time/batch: 56.1814
batch: 53/1000 	    computation time/batch: 56.1969sec	    number of points: 863
	    median distance2db: 1.239776611328125e-05
computation DB time/batch: 56.2459
batch: 54/1000 	    computation time/batch: 56.2615sec	    number of points: 919
	    median distance2db: 1.2636184692382812e-05
computation DB time/batch: 56.2619
batch: 55/1000 	    computation time/batch: 56.2783sec	    number of points: 926
	    median distance2db: 1.633167266845703e-05
computation DB time/batch: 56.0964
batch: 56/1000 	    computation time/batch: 56.1124sec	    number of points: 899
	    median distance2db: 1.0251998901367188e-05
computation DB time/batch: 56.0267
batch: 57/1000 	    computation time/batch: 56.0424sec	    number of points: 896
	    median distance2db: 1.3113021850585938e-05
computation DB tim

computation DB time/batch: 56.1398
batch: 103/1000 	    computation time/batch: 56.1552sec	    number of points: 887
	    median distance2db: 1.6450881958007812e-05
computation DB time/batch: 56.1192
batch: 104/1000 	    computation time/batch: 56.1353sec	    number of points: 902
	    median distance2db: 7.62939453125e-06
computation DB time/batch: 56.0895
batch: 105/1000 	    computation time/batch: 56.1051sec	    number of points: 884
	    median distance2db: 2.4199485778808594e-05
computation DB time/batch: 56.0755
batch: 106/1000 	    computation time/batch: 56.0924sec	    number of points: 947
	    median distance2db: 1.4662742614746094e-05
computation DB time/batch: 56.1327
batch: 107/1000 	    computation time/batch: 56.1506sec	    number of points: 886
	    median distance2db: 9.5367431640625e-06
computation DB time/batch: 56.1415
batch: 108/1000 	    computation time/batch: 56.1580sec	    number of points: 896
	    median distance2db: 2.1457672119140625e-05
computation DB tim

computation DB time/batch: 56.2035
batch: 153/1000 	    computation time/batch: 56.2199sec	    number of points: 894
	    median distance2db: 1.3828277587890625e-05
computation DB time/batch: 56.1415
batch: 154/1000 	    computation time/batch: 56.1576sec	    number of points: 894
	    median distance2db: 7.867813110351562e-06
computation DB time/batch: 56.1729
batch: 155/1000 	    computation time/batch: 56.1898sec	    number of points: 910
	    median distance2db: 9.059906005859375e-06
computation DB time/batch: 56.1842
batch: 156/1000 	    computation time/batch: 56.2001sec	    number of points: 892
	    median distance2db: 1.633167266845703e-05
computation DB time/batch: 56.2173
batch: 157/1000 	    computation time/batch: 56.2339sec	    number of points: 920
	    median distance2db: 1.6450881958007812e-05
computation DB time/batch: 56.2287
batch: 158/1000 	    computation time/batch: 56.2453sec	    number of points: 918
	    median distance2db: 9.417533874511719e-06
computation DB

computation DB time/batch: 56.3025
batch: 203/1000 	    computation time/batch: 56.3190sec	    number of points: 925
	    median distance2db: 8.106231689453125e-06
computation DB time/batch: 56.1885
batch: 204/1000 	    computation time/batch: 56.2045sec	    number of points: 843
	    median distance2db: 1.1563301086425781e-05
computation DB time/batch: 56.2598
batch: 205/1000 	    computation time/batch: 56.2768sec	    number of points: 922
	    median distance2db: 1.6450881958007812e-05
computation DB time/batch: 56.1950
batch: 206/1000 	    computation time/batch: 56.2108sec	    number of points: 840
	    median distance2db: 2.002716064453125e-05
computation DB time/batch: 56.1728
batch: 207/1000 	    computation time/batch: 56.1893sec	    number of points: 901
	    median distance2db: 2.3484230041503906e-05
computation DB time/batch: 56.1725
batch: 208/1000 	    computation time/batch: 56.1889sec	    number of points: 906
	    median distance2db: 2.5510787963867188e-05
computation 

...

computation DB time/batch: 56.0018
batch: 755/1000 	    computation time/batch: 56.0189sec	    number of points: 884
	    median distance2db: 1.3470649719238281e-05
computation DB time/batch: 56.0969
batch: 756/1000 	    computation time/batch: 56.1145sec	    number of points: 907
	    median distance2db: 1.1920928955078125e-05
computation DB time/batch: 55.8541
batch: 757/1000 	    computation time/batch: 55.8709sec	    number of points: 865
	    median distance2db: 1.2516975402832031e-05
computation DB time/batch: 56.2006
batch: 758/1000 	    computation time/batch: 56.2170sec	    number of points: 829
	    median distance2db: 9.298324584960938e-06
computation DB time/batch: 55.9374
batch: 759/1000 	    computation time/batch: 55.9545sec	    number of points: 882
	    median distance2db: 1.2159347534179688e-05
computation DB time/batch: 56.0069
batch: 760/1000 	    computation time/batch: 56.0251sec	    number of points: 929
	    median distance2db: 1.5020370483398438e-05
computation

computation DB time/batch: 55.8779
batch: 805/1000 	    computation time/batch: 55.8960sec	    number of points: 918
	    median distance2db: 1.2159347534179688e-05
computation DB time/batch: 56.0990
batch: 806/1000 	    computation time/batch: 56.1165sec	    number of points: 897
	    median distance2db: 1.3113021850585938e-05
computation DB time/batch: 55.8279
batch: 807/1000 	    computation time/batch: 55.8452sec	    number of points: 849
	    median distance2db: 2.1696090698242188e-05
computation DB time/batch: 55.8329
batch: 808/1000 	    computation time/batch: 55.8505sec	    number of points: 909
	    median distance2db: 1.9550323486328125e-05
computation DB time/batch: 55.9285
batch: 809/1000 	    computation time/batch: 55.9459sec	    number of points: 868
	    median distance2db: 1.4662742614746094e-05
computation DB time/batch: 55.8860
batch: 810/1000 	    computation time/batch: 55.9024sec	    number of points: 839
	    median distance2db: 1.3828277587890625e-05
computatio

computation DB time/batch: 55.8569
batch: 855/1000 	    computation time/batch: 55.8750sec	    number of points: 915
	    median distance2db: 1.621246337890625e-05
computation DB time/batch: 56.1055
batch: 856/1000 	    computation time/batch: 56.1241sec	    number of points: 913
	    median distance2db: 1.919269561767578e-05
computation DB time/batch: 55.9532
batch: 857/1000 	    computation time/batch: 55.9707sec	    number of points: 879
	    median distance2db: 1.7523765563964844e-05
computation DB time/batch: 56.2574
batch: 858/1000 	    computation time/batch: 56.2757sec	    number of points: 916
	    median distance2db: 1.9073486328125e-05
computation DB time/batch: 55.8743
batch: 859/1000 	    computation time/batch: 55.8917sec	    number of points: 900
	    median distance2db: 1.0728836059570312e-05
computation DB time/batch: 55.7728
batch: 860/1000 	    computation time/batch: 55.7902sec	    number of points: 874
	    median distance2db: 7.62939453125e-06
computation DB time/

computation DB time/batch: 56.1209
batch: 905/1000 	    computation time/batch: 56.1389sec	    number of points: 897
	    median distance2db: 8.821487426757812e-06
computation DB time/batch: 56.2451
batch: 906/1000 	    computation time/batch: 56.2633sec	    number of points: 868
	    median distance2db: 1.0371208190917969e-05
computation DB time/batch: 55.8929
batch: 907/1000 	    computation time/batch: 55.9107sec	    number of points: 895
	    median distance2db: 1.8358230590820312e-05
computation DB time/batch: 56.0491
batch: 908/1000 	    computation time/batch: 56.0669sec	    number of points: 928
	    median distance2db: 1.049041748046875e-05
computation DB time/batch: 56.0642
batch: 909/1000 	    computation time/batch: 56.0819sec	    number of points: 891
	    median distance2db: 1.430511474609375e-05
computation DB time/batch: 56.1700
batch: 910/1000 	    computation time/batch: 56.1869sec	    number of points: 858
	    median distance2db: 1.2636184692382812e-05
computation D

computation DB time/batch: 55.8747
batch: 955/1000 	    computation time/batch: 55.8920sec	    number of points: 857
	    median distance2db: 1.537799835205078e-05
computation DB time/batch: 55.8933
batch: 956/1000 	    computation time/batch: 55.9113sec	    number of points: 883
	    median distance2db: 2.014636993408203e-05
computation DB time/batch: 55.8974
batch: 957/1000 	    computation time/batch: 55.9149sec	    number of points: 840
	    median distance2db: 1.0848045349121094e-05
computation DB time/batch: 55.8897
batch: 958/1000 	    computation time/batch: 55.9066sec	    number of points: 833
	    median distance2db: 1.33514404296875e-05
computation DB time/batch: 55.9461
batch: 959/1000 	    computation time/batch: 55.9634sec	    number of points: 862
	    median distance2db: 9.298324584960938e-06
computation DB time/batch: 55.9347
batch: 960/1000 	    computation time/batch: 55.9516sec	    number of points: 840
	    median distance2db: 2.0503997802734375e-05
computation DB 

In [16]:
db_sample_df.to_csv('computed_points_1M.csv')

In [14]:
print(f'computed_points_1M_{10:04d}.csv')

computed_points_1M_0010.csv


In [49]:
print(f'Number of points: {db_sample_df.shape}')

fig = px.scatter_3d(db_sample_df[:1000], x='x0', y='x1', z='x2')
fig.show()

Number of points: (52, 3)


In [ ]:
db_sample_df.to_csv('DB_3d_MNIST.csv')

In [51]:
print(f'Number of points: {db_sample_df.shape}')

fig = px.scatter_3d(db_sample_df[:1000], x='x0', y='x1', z='x2')
fig.show()

Number of points: (106, 3)


In [41]:
embedding_model_multi = nn.Sequential(model_auto.decoder,
                    LambdaLayer(lambda x: x.reshape(-1, 1, 28, 28)),
                    model)

In [44]:
embedding_model_multi.to('cpu')

Sequential(
  (0): Sequential(
    (0): Linear(in_features=3, out_features=12, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=12, out_features=64, bias=True)
    (3): ReLU(inplace=True)
    (4): Linear(in_features=64, out_features=128, bias=True)
    (5): ReLU(inplace=True)
    (6): Linear(in_features=128, out_features=784, bias=True)
    (7): Tanh()
  )
  (1): LambdaLayer()
  (2): Net(
    (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
    (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
    (dropout1): Dropout(p=0.25, inplace=False)
    (dropout2): Dropout(p=0.5, inplace=False)
    (fc1): Linear(in_features=9216, out_features=128, bias=True)
    (fc2): Linear(in_features=128, out_features=10, bias=True)
  )
)

In [45]:
db_tensor = torch.tensor(db_sample_df.values)
db_tensor_logit = embedding_model_multi(db_tensor)
max_top2 = torch.topk(db_tensor_logit, 2).indices.max(axis=1)
db_class = {}
for digit_class in range(10):
    db_class[digit_class] = {}
    db_class[digit_class]['tensor'] = max_top2.values == digit_class
    db_class[digit_class]['idx'] = torch.nonzero(db_class[digit_class]['tensor'])

In [46]:
db_sample_class_df = pd.DataFrame(columns=['class'])

for digit_class in range(10):
    db_sample_class_df = db_sample_class_df.append(pd.DataFrame(db_class[digit_class]['idx'][:,0].shape[0]*[digit_class], columns=['class'], index=db_class[digit_class]['idx'][:,0].numpy()))

In [ ]:
db_sample_normal_df = pd.DataFrame(columns = ["dx"+str(i) for i in range(3)])

for digit_class in range(10):
    x = db_tensor[db_class[digit_class]['tensor']]
    if x.shape[0] != 0:
        x.requires_grad = False
        delta = torch.zeros_like(x, requires_grad=True)
        loss = torch.sum(embedding_model_multi(x + delta)[:, digit_class])
        loss.backward()
        x_gradient = delta.grad.detach().numpy()
        x_norm = x_gradient/np.sqrt((x_gradient**2).sum(axis=1, keepdims=True))

        db_sample_normal_df = db_sample_normal_df.append(
            pd.DataFrame(data=x_norm,
                columns = ["dx"+str(i) for i in range(3)],
                index=db_class[digit_class]['idx'].numpy()[:,0])
        )

In [ ]:
db_mesh_input = pd.concat([db_sample_df, db_sample_normal_df, db_sample_class_df], axis=1, join='inner')